In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('nlp').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 22:42:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/25 22:42:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/25 22:42:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/25 22:42:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [3]:
sen_df = spark.createDataFrame(
    data=[
        (0,'Hi I heard about Spark'),
        (1, 'I wish java could use case classes'),
        (2, 'Logistic,regression,models,are,neat')
    ],
    schema=['id','sentence']
)

In [5]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')
count_tokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(sen_df)
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [7]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [8]:
rg_tokenized = regex_tokenizer.transform(sen_df)
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



### Removed StopWords

In [9]:
from pyspark.ml.feature import StopWordsRemover

sentenceDataFrame = spark.createDataFrame(
    data=[
        (0, ['I', 'saw', 'the', 'green', 'horse']),
        (1, ['Mary', 'had', 'a', 'little', 'lamb'])
    ],
    schema=['id','tokens']
)
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



### N-Gram

In [10]:
from pyspark.ml.feature import NGram

wordDataFrame = spark.createDataFrame([
    (0,['Hi', 'I', 'heard', 'about', 'spark']),
    (1,['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
    (2,['Logistic', 'regression', 'models', 'are', 'neat'])
],['id', 'words'])
ngram = NGram(n=2, inputCol='words', outputCol='grams')
ngram.transform(wordDataFrame).select('grams').show(truncate=False) 

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+

